In [26]:
import pandas as pd
import numpy as np

In [27]:
all_vccsid_strm_1 = pd.read_stata("C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\bias\\all_observations.dta")\
.loc[:,['vccsid','strm']].drop_duplicates()
all_vccsid_strm_2 = pd.read_stata("C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\bias\\all_vccsid_strm.dta")
all_vccsid_strm = all_vccsid_strm_1.merge(all_vccsid_strm_2, on=['vccsid', 'strm'], how="inner")
all_vccsid_strm.shape

(1941502, 3)

In [28]:
sum(all_vccsid_strm.first_strm == all_vccsid_strm.strm), sum(all_vccsid_strm.first_strm == all_vccsid_strm.strm)/all_vccsid_strm.shape[0]

(310228, 0.15978762834135632)

In [29]:
all_vccsid_strm.loc[:,'first_ind'] = all_vccsid_strm.apply(lambda x: x.strm == x.first_strm, axis=1).astype(int)
all_vccsid_strm = all_vccsid_strm.drop(['first_strm'], axis=1)

In [30]:
from collections import Counter
Counter(all_vccsid_strm.first_ind)

Counter({0: 1631274, 1: 310228})

In [31]:
d1 = pd.read_stata('C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\bias\\age_gpa_earn.dta')
d2 = pd.read_stata('C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\bias\\four_additional_predictors.dta')
d3 = pd.read_stata('C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\bias\\twelve_additional_predictors.dta')
d4 = pd.read_stata('C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\bias\\trend_predictors.dta')
d5 = pd.read_stata('C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\bias\\ever_dually_enrolled.dta')

In [32]:
merged = all_vccsid_strm.merge(d1, how='inner', on=['vccsid','strm']).\
merge(d2, how='inner', on=['vccsid','strm']).\
merge(d3, how='inner', on=['vccsid','strm']).\
merge(d4, how='inner', on=['vccsid','strm']).\
merge(d5, how='inner', on=['vccsid','strm'])

In [33]:
merged.shape

(1941502, 23)

In [34]:
merged_part1 = merged[merged.first_ind == 1]
merged_part2 = merged[merged.first_ind == 0]

In [38]:
merged_part1 = merged_part1.fillna(0)

In [39]:
pd.isnull(merged_part2).any()

vccsid                  False
strm                    False
first_ind               False
cum_gpa                  True
cum_cred_earn           False
age                     False
num_of_prior_terms      False
pct_stopped             False
pct_withdrawn           False
pct_incomplete          False
pct_dev                 False
prop_comp_sd             True
overall_prop_comp        True
first_strm              False
crnt_enrl_intensity     False
enrl_intensity          False
has_term_gpa_1          False
term_gpa_1              False
has_term_gpa_2          False
term_gpa_2              False
gpa_trend               False
enrl_intensity_trend    False
ever_dual               False
dtype: bool

In [40]:
impute_list_1 = set(["cum_gpa", "overall_prop_comp", "prop_comp_sd"])

In [41]:
def impute(train, test):
    for p in impute_list_1:
        avg_p = np.nanmean(train[p])
        train.loc[:,p] = train.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
        test.loc[:,p] = test.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
    return train, test               

In [42]:
train_df, test_df = impute(merged_part2[merged_part2.strm < 2192], merged_part2[merged_part2.strm >= 2192])
merged_part2_new = pd.concat([train_df, test_df])

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [43]:
merged_new = pd.concat([merged_part1, merged_part2_new])

In [46]:
merged_new.to_csv("C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\bias\\term_specific_predictors.csv", index=False)

In [47]:
assert pd.isnull(merged_new).any().any() == False
merged_new.shape

(1941502, 23)